# Python Testing Script
Use this template to create a Python testing script that imports your CRUD Python module to call and test the create and read instances of CRUD functionality. 
### 1. Import your CRUD Python module
### 2. Instantiate an instance of your class
### 3. Use your create function create a new record in the aac database
### 4. Use your read funtion to return records from the aac database 
### 5. Use your update function to update an existing record in the aac database
### 6. Use your delete function to delete an existing record in the aac database

<div class="alert alert-block alert-success"><span style="color:black;"><b>Tip: </b>You should have created test functionality for items 1 thru 4 during Module Four Milestone, copy over your existing test script code and expand it here to call and test the Update and Delete functionality.</span></div>

In [2]:
# --- Setup: run this once at the top of the notebook ---
import importlib, CRUD_Python_Module
importlib.reload(CRUD_Python_Module)
from CRUD_Python_Module import AnimalShelter
from pprint import pprint

shelter = AnimalShelter()          # <— creates the client you need
TEST_ID = "CS340TEST123"           # used by CRUD demo
_ = shelter.delete({"animal_id": TEST_ID}, many=True)  # cleanup if re-running


In [3]:
import csv, itertools, os

path = "../datasets/aac_shelter_outcomes.csv"
assert os.path.exists(path), f"CSV not found at {path}"

def chunks(iterable, size=1000):
    it = iter(iterable)
    while True:
        batch = list(itertools.islice(it, size))
        if not batch:
            break
        yield batch

rows = []
with open(path, newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for r in reader:
        for k, v in r.items():
            if v == "":
                r[k] = None
        rows.append(r)

# Only insert if empty to avoid duplicates
before = shelter.col.count_documents({})
inserted_total = 0
if before == 0:
    for batch in chunks(rows, 1000):
        res = shelter.col.insert_many(batch)
        inserted_total += len(res.inserted_ids)

after = shelter.col.count_documents({})
print("Docs before:", before)
print("Inserted now:", inserted_total)
print("Docs after:", after)


Docs before: 20000
Inserted now: 0
Docs after: 20000


In [4]:
print("DB:", shelter.db.name)
print("Collection:", shelter.col.name)
print("Total docs:", shelter.col.count_documents({}))
print("Ping:", shelter.db.command("ping"))  # expect {'ok': 1.0}


DB: AAC
Collection: animals
Total docs: 20000
Ping: {'ok': 1}


In [5]:
new_animal = {
    "animal_id": "Z999999",
    "name": "Shadow",
    "animal_type": "Dog",
    "breed": "Mixed",
    "color": "Black",
    "outcome_type": "Adoption"
}
shelter.create(new_animal)
print("Inserted Shadow:", shelter.col.find_one({"animal_id": "Z999999"}))


Inserted Shadow: {'_id': ObjectId('68e15a77305768716c6ed280'), 'animal_id': 'Z999999', 'name': 'Shadow', 'animal_type': 'Dog', 'breed': 'Mixed', 'color': 'Black', 'outcome_type': 'Adoption'}


In [6]:
results = shelter.read({"animal_type": "Dog"}, limit=3)
for r in results.detail:
    print(r)


{'_id': '68e13f89745741b126356529', 'rec_num': '3', 'age_upon_outcome': '2 years', 'animal_id': 'A716330', 'animal_type': 'Dog', 'breed': 'Chihuahua Shorthair Mix', 'color': 'Brown/White', 'date_of_birth': '2013-11-18', 'datetime': '2015-12-28 18:43:00', 'monthyear': '2015-12-28T18:43:00', 'name': 'Frank', 'outcome_subtype': None, 'outcome_type': 'Adoption', 'sex_upon_outcome': 'Neutered Male', 'location_lat': '30.7595748121648', 'location_long': '-97.5523753807133', 'age_upon_outcome_in_weeks': '110.111408730159'}
{'_id': '68e13f89745741b12635652b', 'rec_num': '5', 'age_upon_outcome': '2 years', 'animal_id': 'A691584', 'animal_type': 'Dog', 'breed': 'Labrador Retriever Mix', 'color': 'Tan/White', 'date_of_birth': '2012-11-06', 'datetime': '2015-05-30 13:48:00', 'monthyear': '2015-05-30T13:48:00', 'name': 'Luke', 'outcome_subtype': None, 'outcome_type': 'Return to Owner', 'sex_upon_outcome': 'Neutered Male', 'location_lat': '30.7104815618433', 'location_long': '-97.562297435286', 'age_

In [7]:
update_result = shelter.col.update_one({"animal_id": "Z999999"}, {"$set": {"color": "Brown"}})
print("Modified count:", update_result.modified_count)
print("Updated document:", shelter.col.find_one({"animal_id": "Z999999"}))


Modified count: 1
Updated document: {'_id': ObjectId('68e15a77305768716c6ed280'), 'animal_id': 'Z999999', 'name': 'Shadow', 'animal_type': 'Dog', 'breed': 'Mixed', 'color': 'Brown', 'outcome_type': 'Adoption'}


In [8]:
delete_result = shelter.col.delete_one({"animal_id": "Z999999"})
print("Deleted count:", delete_result.deleted_count)
print("Verify delete:", shelter.col.find_one({"animal_id": "Z999999"}))


Deleted count: 1
Verify delete: None


In [9]:
# 1) Check with your module (should be an empty list)
confirm_with_module = shelter.read({"animal_id": "Z999999"})
print("Module read:", confirm_with_module.detail)   # expect []

# 2) Check with count (should be 0)
print("Count:", shelter.col.count_documents({"animal_id": "Z999999"}))  # expect 0

# 3) Check with find_one (should be None)
print("find_one:", shelter.col.find_one({"animal_id": "Z999999"}))      # expect None


Module read: []
Count: 0
find_one: None
